# Notebook for Data Exploration of Aachen dataset

In [1]:
import numpy as np
import pandas as pd

from Load_and_Preprocess_Aachen import preprocess_aachen_dataset



In [2]:
data = preprocess_aachen_dataset()

TypeError: preprocess_aachen_dataset() missing 1 required positional argument: 'file_path'